<a href="https://colab.research.google.com/github/Chandan062311/Augumentation/blob/main/Augmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget https://storage.googleapis.com/tensorflow-1-public/course2/cats_and_dogs_filtered.zip


--2024-03-24 16:35:29--  https://storage.googleapis.com/tensorflow-1-public/course2/cats_and_dogs_filtered.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 209.85.200.207, 209.85.234.207, 142.251.180.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|209.85.200.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 68606236 (65M) [application/zip]
Saving to: ‘cats_and_dogs_filtered.zip’

cats_and_dogs_filte 100%[===================>]  65.43M  54.7MB/s    in 1.2s    

2024-03-24 16:35:31 (54.7 MB/s) - ‘cats_and_dogs_filtered.zip’ saved [68606236/68606236]



In [ ]:
import os
import zipfile

In [ ]:
zip_ref = zipfile.ZipFile("/content/cats_and_dogs_filtered.zip","r")
zip_ref.extractall("tmp/")
zip_ref.close()

In [ ]:
base_dir = '/content/tmp/cats_and_dogs_filtered'
train_dir = os.path.join(base_dir,'train')
validation_dir = os.path.join(base_dir,'validation')

In [ ]:
train_cats_dir = os.path.join(train_dir,'cats')
train_dogs_dir = os.path.join(train_dir,'dogs')

In [ ]:
validation_cats_dir = os.path.join(validation_dir,'cats')
validation_dogs_dir = os.path.join(validation_dir,'dogs')

In [ ]:
import tensorflow as tf
from tensorflow.keras.optimizers import RMSprop

In [ ]:
def create_model():
  model=tf.keras.models.Sequential([
      tf.keras.layers.Conv2D(32,(3,3),activation='relu',input_shape=(150,150,3)),
      tf.keras.layers.MaxPooling2D(2,2),
      tf.keras.layers.Conv2D(64,(3,3),activation='relu'),
      tf.keras.layers.MaxPooling2D(2,2),
      tf.keras.layers.Conv2D(128,(3,3),activation='relu'),
      tf.keras.layers.MaxPooling2D(2,2),
      tf.keras.layers.Conv2D(128,(3,3),activation='relu'),
      tf.keras.layers.MaxPooling2D(2,2),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(512,activation='relu'),
      tf.keras.layers.Dense(1,activation='sigmoid')

  ])
  model.compile(loss='binary_crossentropy',optimizer=RMSprop(learning_rate=1e-4),metrics=['accuracy'])
  return model

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=40,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   fill_mode='nearest'
                                   )
test_datagen = ImageDataGenerator(rescale=1./255,
                                  rotation_range=40,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   fill_mode='nearest')

In [ ]:
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150,150),
    batch_size=20,
    class_mode = 'binary'
)

Found 2000 images belonging to 2 classes.


In [ ]:
validation_generator = test_datagen.flow_from_directory(
    validation_dir,
    target_size=(150,150),
    batch_size=20,
    class_mode = 'binary'
)

Found 1000 images belonging to 2 classes.


In [ ]:
EPOCHS = 20

In [ ]:
model = create_model()



In [ ]:
history = model.fit(
    train_generator,
    steps_per_epoch=100,
    epochs=EPOCHS,
    validation_data = validation_generator,
    validation_steps=50,
    verbose=2
)

Epoch 1/20
100/100 - 172s - loss: 0.6931 - accuracy: 0.4970 - val_loss: 0.6864 - val_accuracy: 0.5860 - 172s/epoch - 2s/step
Epoch 2/20
100/100 - 166s - loss: 0.6852 - accuracy: 0.5400 - val_loss: 0.6676 - val_accuracy: 0.5740 - 166s/epoch - 2s/step
Epoch 3/20
100/100 - 166s - loss: 0.6738 - accuracy: 0.5740 - val_loss: 0.6572 - val_accuracy: 0.6000 - 166s/epoch - 2s/step
Epoch 4/20
100/100 - 167s - loss: 0.6674 - accuracy: 0.5755 - val_loss: 0.6671 - val_accuracy: 0.5530 - 167s/epoch - 2s/step
Epoch 5/20
100/100 - 166s - loss: 0.6561 - accuracy: 0.5915 - val_loss: 0.6374 - val_accuracy: 0.6260 - 166s/epoch - 2s/step
Epoch 6/20
100/100 - 173s - loss: 0.6485 - accuracy: 0.6115 - val_loss: 0.6284 - val_accuracy: 0.6540 - 173s/epoch - 2s/step
Epoch 7/20
100/100 - 167s - loss: 0.6450 - accuracy: 0.6140 - val_loss: 0.6444 - val_accuracy: 0.6080 - 167s/epoch - 2s/step
Epoch 8/20
100/100 - 146s - loss: 0.6307 - accuracy: 0.6305 - val_loss: 0.6067 - val_accuracy: 0.6690 - 146s/epoch - 1s/step


In [18]:
import matplotlib.pyplot as plt

def plot_loss_acc(history):
  '''Plots the training and validation loss and accuracy from a history object'''
  acc = history.history['accuracy']
  val_acc = history.history['val_accuracy']
  loss = history.history['loss']
  val_loss = history.history['val_loss']

  epochs = range(len(acc))

  plt.plot(epochs, acc, 'bo', label='Training accuracy')
  plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
  plt.title('Training and validation accuracy')

  plt.figure()

  plt.plot(epochs, loss, 'bo', label='Training Loss')
  plt.plot(epochs, val_loss, 'b', label='Validation Loss')
  plt.title('Training and validation loss')
  plt.legend()

  plt.show()